In [ ]:
import os
import cv2
import time
import numpy as np
import scipy.fftpack as fftpack
from scipy import signal
from imutils import face_utils
import imutils
import dlib

In [ ]:
def build_laplacian_pyramid(img):
    gaussian_pyramid = cv2.pyrDown(img)

    upsampled = cv2.pyrUp(gaussian_pyramid)
    (height, width, depth) = upsampled.shape
    gaussian_pyramid = cv2.resize(gaussian_pyramid, (height, width))

    return cv2.subtract(gaussian_pyramid, upsampled)

In [ ]:
class FaceDetection(object):
    def __init__(self):
        self.detector = dlib.get_frontal_face_detector()

    def face_detect(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = self.detector(gray, 0)
        
        if len(rects)>0:
            (x, y, w, h) = face_utils.rect_to_bb(rects[0])
            if y >= 0:
                return (x, y, w, h)
        
        return None

In [ ]:
def read_video(path, Detector):
    cap = cv2.VideoCapture(path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    video_frames = []
    loc = None

    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            break
        
#         if img is rotated
#         img = cv2.resize(img, (500, 500))
#         img = imutils.rotate(img, 180)
        
        if loc is None:
            loc = Detector.face_detect(img)
        
        if loc is not None:
            (x, y, w, h) = loc
            img = img[y:y+h, x:x+w]
            video_frames.append(build_laplacian_pyramid(img))
    
    cap.release()
    
    return np.array(video_frames), fps

In [ ]:
def fft_filter(video, freq_min, freq_max, fps):
    fft = fftpack.fft(video, axis=0)
    frequencies = fftpack.fftfreq(video.shape[0], d=1.0 / fps)
    bound_low = (np.abs(frequencies - freq_min)).argmin()
    bound_high = (np.abs(frequencies - freq_max)).argmin()
    fft[:bound_low] = 0
    fft[bound_high:-bound_high] = 0
    fft[-bound_low:] = 0
    iff = fftpack.ifft(fft, axis=0)

    return fft, frequencies

In [ ]:
def find_heart_rate(fft, freqs, freq_min, freq_max):
    fft_maximums = []

    for i in range(fft.shape[0]):
        if freq_min <= freqs[i] <= freq_max:
            fftMap = abs(fft[i])
            fft_maximums.append(fftMap.max())
        else:
            fft_maximums.append(0)

    peaks, properties = signal.find_peaks(fft_maximums)
    max_peak = -1
    max_freq = 0

    # Find frequency with max amplitude in peaks
    for peak in peaks:
        if fft_maximums[peak] > max_freq:
            max_freq = fft_maximums[peak]
            max_peak = peak

    return freqs[max_peak] * 60

In [ ]:
Detector = FaceDetection()

freq_min = 1
freq_max = 1.8

base = '../data'
# file_name : fileNumber_tureHR.avi

In [ ]:
for file in os.listdir(os.path.join(base)):
    try:
        video, fps = read_video(os.path.join(base, file), Detector)
        fft, frequencies = fft_filter(video, freq_min, freq_max, fps)
        heart_rate = find_heart_rate(fft, frequencies, freq_min, freq_max)

        file_name = file.split('.')[0]
        print(f"{file_name.split('_')[0]}: {file_name.split('_')[1]} {heart_rate},")
    except: # if detected face is zero
        print(f"{file} is Error")
        continue

# check the detected face

In [ ]:
loc = None
base = '../data'

for file in os.listdir(base):
    print(file)
    cap = cv2.VideoCapture(os.path.join(base, file))

    while cap.isOpened():
        ret, img = cap.read()
        if not ret:
            break
        
        if loc is None:
            loc = Detector.face_detect(img)
        
        if loc is not None:
            (x, y, w, h) = loc
            img = img[y:y+h, x:x+w]
            
        cv2.imshow('img', img)
        if cv2.waitKey(33) > 0: 
            break
        
        time.sleep(0.1)
    
    print("end")
    cv2.destroyAllWindows()
    cap.release()
    
    time.sleep(5)